In [ ]:
import pandas as pd
import geopandas as gpd
import os
import contextily as cx
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch
import gma.extend.mapplottools as mpt
import matplotlib as mpl

def join_dict(fi, dir,jdict):
    if 'citibike' in dir:
        jdict['k1'] = 'name'
        if 'ends' in fi:
            jdict['k2'] = 'end_station_name'
            jdict['flag'] = 1
        else:
            jdict['k2'] = 'start_station_name'
            jdict['flag'] = 0
    if 'TLC' in dir:
        jdict['k1'] = 'location_i'
        if 'Dropoffs' in fi:
            jdict['k2'] = 'DOLocationID'
            jdict['flag'] = 1
        else:
            jdict['k2'] = 'PULocationID'
            jdict['flag'] = 0
        
    if 'Turnstile' in dir:
        jdict['k1'] = 'sta_line'
        jdict['k2'] = 'station_line'
        if 'Entries' in fi:
            jdict['flag'] = 0
        else:
            jdict['flag'] = 1

    return jdict

    
def att_join(files, dir, shp, ts, gdfs):
    
    for fi in files:#this files are for a hourly-grouped fig
        jdict = {}
        jdict = join_dict(fi, dir,jdict)
        
        t = fi[4:6]
        
        k1 = jdict['k1']
        k2 = jdict['k2']
        flag = jdict['flag']
        
        dfi = pd.read_csv(dir+fi)
        gdfi = gpd.GeoDataFrame(dfi.merge(shp, left_on=k2, right_on=k1))
        
        gdfs[t,flag] = gdfi
        
    return gdfs




def get_vec(files, dir, shp,ts,evt,cols):
    areas = gpd.read_file(r'NYC_taxi_zones.shp').to_crs(3857)
    # print(areas)
    z_cols =  [i.replace('r', 'az') for i in cols]
    gdfs = {}
    gdfs = att_join(files, dir, shp, ts, gdfs)
    
    # print(gdfs)
    
    vects={}
    # for t in ts:
    for f in [0,1]:#f=0 means origin data, f=1 means destination data
        if f==0:
            vects_O = pd.DataFrame()
        else:
            vects_D = pd.DataFrame()
        
        for t in ts:
            i = int(int(t)/4)#x loc for a subplot
            gdfi = gdfs[t,f]
            # print(gdfi)
            for i in z_cols:
                gdfi[i.replace('az','r')].loc[gdfi[i]<3] = 0
            gdfi = gdfi.to_crs('EPSG:3857')
                # print(gdfi)
            if ('citibike' in dir) or ("Turnstile" in dir):
                joined_gdf = gpd.sjoin(gdfi, areas, how='inner', op='within')
                    # print(joined_gdf)
                vec = joined_gdf.groupby('zone')[cols].mean()
                vec.reset_index(inplace=True)

            else:
                vec = gdfi[cols+['zone']]
                # vec.set_index('zone',inplace=True)
            
            # vec = vec/(vec.max().max()-vec.min().min())
            vec2 = vec.melt(id_vars=['zone'], value_vars=cols, var_name='datetime', value_name='resid')
            vec2['datetime'] = vec2['datetime']+':'+t
            # print(vec2)
            if f==0:
                vects_O = vects_O.append(vec2,ignore_index=True)
            else:
                vects_D = vects_D.append(vec2,ignore_index=True)
    
    
    vects_O = vects_O.fillna(0)
    vects_D = vects_D.fillna(0)
    
    return vects_O, vects_D


def add_miss(df,cols,ts):
    areas = gpd.read_file(r'NYC_taxi_zones.shp').to_crs(3857)
    tmp = pd.DataFrame(areas['zone'])
    
    tmp['datetime'] = [[(x+':'+y) for y in ts for x in cols]]*len(tmp)
    tmp = tmp.explode('datetime') 
    
    tmp['trans'] = [['FHV','Taxi','Subway','Citibike']]*len(tmp)
    tmp = tmp.explode('trans') 
    idx = tmp.set_index(['zone','datetime','trans']).index
    
    df = df.set_index(['zone','datetime','trans'])
    mis = [i for i in idx if i not in df.index]
    df = pd.concat([df,pd.DataFrame(index=mis)])
    
    df.reset_index(inplace=True)
    df = df.fillna(0)
    df = df.pivot_table(index=['zone','datetime'], columns='trans', values='resid', aggfunc='first')
    df.reset_index(inplace=True)
    # df.rename(columns = {'trans':'index'})
    
    df['datetime'] = '20'+df['datetime'].str.split('_').str[1]
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y%m%d:%H')
    
    return df
    
    
def get_all_vects(dirs,G_dir,events,ts):
    all_vects_O={}
    all_vects_D={}
    
    for evt in events:
        if evt=='Aug':
            key='Henri'
            cols = ['r_210821','r_210822','r_210823']
            
        elif evt=='Sep':
            key = 'Ida'
            cols = ['r_210901','r_210902']
            
        elif evt=='Jul':
            key = 'Elsa'
            cols = ['r_210708','r_210709']
            
        elif evt=='Oct':
            key = 'Nor'
            cols = ['r_211025','r_211026']
   
        all_vects_O[key]=pd.DataFrame()
        all_vects_D[key]=pd.DataFrame()
        
        for dir in dirs:
            dir = main_dir + dir
            files = [i for i in os.listdir(dir) if os.path.splitext(i)[1] == '.csv' and i[4:6] in ts]
            efiles = [i for i in files if evt in i]
            
            if 'TLC' in dir:
                shp = gpd.read_file(G_dir+r'NYC Taxi Zones\NYC_taxi_zones.shp').to_crs(3857)
                HVFHV = [i for i in efiles if 'HVFHV' in i]
                Taxi = [i for i in efiles if 'of Taxi' in i]
                O,D = get_vec(HVFHV, dir, shp,ts,evt,cols)
                O['trans'] = 'FHV'
                D['trans'] = 'FHV'
                all_vects_O[key] = all_vects_O[key].append(O,ignore_index=True)
                all_vects_D[key] = all_vects_D[key].append(D,ignore_index=True)
                
                OO,DD = get_vec(Taxi, dir, shp,ts,evt,cols)
                OO['trans'] = 'Taxi'
                DD['trans'] = 'Taxi'
                all_vects_O[key] = all_vects_O[key].append(OO,ignore_index=True)
                all_vects_D[key] = all_vects_D[key].append(DD,ignore_index=True)

            if 'Turnstile' in dir:
                shp = gpd.read_file(G_dir+r'Subway\Subway_and_Path_stations.shp').to_crs(3857)
                stations = [i for i in efiles if "Station's" in i]
                O,D = get_vec(stations, dir, shp,ts,evt,cols)
                O['trans'] = 'Subway'
                D['trans'] = 'Subway'
                all_vects_O[key] = all_vects_O[key].append(O,ignore_index=True)
                all_vects_D[key] = all_vects_D[key].append(D,ignore_index=True)

            if 'citibike' in dir:
                shp = gpd.read_file(G_dir+r'Bicycle\citibike_stations.shp').to_crs(3857)
                O,D = get_vec(efiles, dir, shp,ts,evt,cols)
                O['trans'] = 'Citibike'
                D['trans'] = 'Citibike'
                all_vects_O[key] = all_vects_O[key].append(O,ignore_index=True)
                all_vects_D[key] = all_vects_D[key].append(D,ignore_index=True)

            
        all_vects_O[key] = add_miss(all_vects_O[key],cols,ts)
        all_vects_D[key] = add_miss(all_vects_D[key],cols,ts)
    
        
    return all_vects_O,all_vects_D



    
    






        
    

            


In [2]:
main_dir = r""
dirs = [r'\TLC\result\without holiday/',r'\Turnstile\result\without holiday/',r'\citibike\result\without holiday/']
G_dir = r'\Geo_shps/'
    
ts = ['00','04','08','12','16','20']
events = ['Aug','Sep','Jul','Oct']

all_vects_O,all_vects_D = get_all_vects(dirs,G_dir,events,ts)
    
print(all_vects_O['Henri'])



trans                     zone            datetime  Citibike       FHV  \
0      Allerton/Pelham Gardens 2021-08-21 00:00:00   0.00000  0.000000   
1      Allerton/Pelham Gardens 2021-08-21 04:00:00   0.00000  0.000000   
2      Allerton/Pelham Gardens 2021-08-21 08:00:00   0.00000  0.000000   
3      Allerton/Pelham Gardens 2021-08-21 12:00:00   0.00000  0.000000   
4      Allerton/Pelham Gardens 2021-08-21 16:00:00   0.00000  0.000000   
...                        ...                 ...       ...       ...   
4675            Yorkville West 2021-08-23 04:00:00   0.00000 -4.726777   
4676            Yorkville West 2021-08-23 08:00:00   0.00000  0.000000   
4677            Yorkville West 2021-08-23 12:00:00  -0.77521  0.000000   
4678            Yorkville West 2021-08-23 16:00:00   0.00000  0.000000   
4679            Yorkville West 2021-08-23 20:00:00   0.00000  0.000000   

trans  Subway  Taxi  
0         0.0   0.0  
1         0.0   0.0  
2         0.0   0.0  
3         0.0   0.0  
4

In [3]:
from tslearn.clustering import TimeSeriesKMeans
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def cluster(df,n_clusters, key):
    df.sort_values(by=['zone', 'datetime'], inplace=True)

    # scaler_standard = StandardScaler()
    # df['FHV'] = scaler_standard.fit_transform(df[['FHV']])
    # df['Taxi'] = scaler_standard.fit_transform(df[['Taxi']])
    # df['Subway'] = scaler_standard.fit_transform(df[['Subway']])
    # df['Citibike'] = scaler_standard.fit_transform(df[['Citibike']])
    print(df)
    result=pd.DataFrame()
    result['zone'] = df['zone'].unique()
    # print(df)
    data = df[['FHV','Taxi','Subway','Citibike']].values.reshape(len(result),-1)
    # print(data)


    # scaler_minmax = MinMaxScaler()
    # data = scaler_minmax.fit_transform(data)
    # '''

    # n_clusters = 4

    # KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    cluster_labels = kmeans.fit_predict(data)
    
    result['cluster'] = cluster_labels
    
    # cluster centers
    cluster_centers = kmeans.cluster_centers_
    
    num_time_periods = len(df[df['zone']==df['zone'].unique()[0]])
    num_features = 4
    centers = pd.DataFrame(cluster_centers).T
    
    feature = pd.DataFrame()
    feature['datetime'] = df['datetime'].unique()
    feature['datetime'] = feature['datetime'].dt.strftime('%m-%d %H')
    # feature.reset_index(inplace=True)
    feature['trans'] = [['FHV','Taxi','Subway','Citibike']]*len(feature)
    
    
    feature = feature.explode('trans')
    feature.reset_index(inplace=True)
    centers['idx'] = feature['datetime']+'_'+feature['trans']
    
    
    centers.to_csv(r'\avg residuals_all modes\dt_anomaly_cluster/'+key+'_cluster.csv',index=False)
   
   
    # 使用肘部法则选择最佳的簇数量
    inertia = []
    silhouette_scores = []
    max_clusters = 20  # 设置最大簇的数量
    
     # 计算真实数据的距离矩阵
    dist_matrix = pairwise_distances(pd.DataFrame(data), metric='euclidean')
   
    # 初始化一个列表来存储Gap Statistics
    gap_stats = []
    
    for n_clusters in range(1, max_clusters + 1):
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        kmeans.fit(data)
        inertia.append(kmeans.inertia_)
        if n_clusters > 1:
            silhouette_scores.append(silhouette_score(data, kmeans.labels_))
            
            # 计算聚类后数据的距离矩阵
            
            cluster_dist_matrix = pairwise_distances(kmeans.cluster_centers_, metric='euclidean')
            
            # 计算Gap Statistics
            
            gap = np.nanmean(np.isfinite(np.log(cluster_dist_matrix))) - np.nanmean(np.isfinite(np.log(dist_matrix)))
            # print(gap)
            gap_stats.append(gap)
            

    # 绘制肘部法则图
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, max_clusters + 1), inertia, marker='o')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Inertia')
    plt.title('Elbow Method')

    plt.subplot(1, 2, 2)
    plt.plot(range(2, max_clusters + 1), silhouette_scores, marker='o')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Silhouette Score')
    plt.title('Silhouette Score')

    plt.tight_layout()
    plt.show()
    
    
    # print(gap_stats)
    # 绘制Gap Statistics图
    plt.plot(range(2, max_clusters + 1), gap_stats, marker='o')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Gap Statistics')
    plt.title('Gap Statistics')
    plt.show()
    #'''
    
        

    
    
    return result
    
    


In [ ]:
# import fiona
# print(fiona.supported_drivers)
pts = gpd.read_file(r'\Geo_shps\NYC Taxi Zones\NYC_taxi_zones_centroid.shp')
# print(pts)
for key in ['Elsa','Henri','Ida','Nor']:
    if key in ['Henri','Ida']:
        n_clusters = 5
    else:
        n_clusters = 4
    dfO = all_vects_O[key]
    dfD = all_vects_D[key]
    
    # dis = cal_distance(dfO)
    # dis.to_csv(r'D:\【Columbia】\project\data\avg residuals_all modes\dt_anomaly_cluster/'+key+'_dt_distance_O.csv',index=False)
    # break
    c_dfo = cluster(dfO, n_clusters, key+'_O')
    if 'Elsa' in key:
        c_dfd = cluster(dfD, 3, key+'_D')
    else:
        c_dfd = cluster(dfD, n_clusters, key+'_D')
    
    
    dfO.to_csv(r'\avg residuals_all modes\dt_anomaly_cluster/'+key+'_dt_anomalies_O.csv',index=False)
    c_dfo.to_csv(r'\avg residuals_all modes\dt_anomaly_cluster/'+key+'_dt_anomalies_O_clu.csv',index=False)
    
    dfD.to_csv(r'\avg residuals_all modes\dt_anomaly_cluster/'+key+'_dt_anomalies_D.csv',index=False)
    c_dfd.to_csv(r'\avg residuals_all modes\dt_anomaly_cluster/'+key+'_dt_anomalies_D_clu.csv',index=False)
    
    gdfO = gpd.GeoDataFrame(c_dfo.merge(pts, left_on='zone', right_on='zone'))
    gdfO.to_file(r'\Geo_shps\dt_anomaly.gpkg', layer=key+'_dt_anomalies_O',driver='GPKG')
    
    gdfD = gpd.GeoDataFrame(c_dfd.merge(pts, left_on='zone', right_on='zone'))
    gdfD.to_file(r'\Geo_shps\dt_anomaly.gpkg', layer=key+'_dt_anomalies_D',driver='GPKG')



In [17]:
def cal_distance(df):
    #     data=pd.DataFrame()
    
    df.sort_values(by=['zone', 'datetime'], inplace=True)
    print(df)
    result=pd.DataFrame()
    result['zone'] = df['zone'].unique()
    means = df[['FHV','Taxi','Subway','Citibike']].mean()
    
    df2 = df.copy()
    
    df2['FHV'] = (df2['FHV'] - means['FHV'])**2
    # df2['FHV'] = df2['FHV'].pow(2)
    df2['Taxi'] = (df2['Taxi'] - means['Taxi'])**2
    df2['Subway'] = (df2['Subway'] - means['Subway'])**2    
    df2['Citibike'] = (df2['Citibike'] - means['Citibike'])**2
    
    # print(df)
    df2['distance'] = df2[['FHV','Taxi','Subway','Citibike']].sum(axis=1)
    df2['distance'] = df2['distance'].round(3).astype('float')
    # print(df)
    return df2
    
#     for k,dfi in groups:
#         dict={}
        
#         for i in ['FHV','Taxi','Subway','Citibike']:
#             dict[i] = dfi[i].values
            
#         dict['zone']=[k]*len(dfi)
        
#         data = data.append(dict,ignore_index=True)
#     print(data)
    

In [18]:
for key in ['Elsa','Henri','Ida','Nor']:
    df_o = all_vects_O[key]
    df_d = all_vects_D[key]
    
    diso = cal_distance(df_o)
    disd = cal_distance(df_d)
    
    diso_sum = diso.groupby('zone').sum().reset_index()#['zone','distance']
    disd_sum = disd.groupby('zone').sum().reset_index()#['zone','distance']
    
    print(diso_sum)
    
    diso_sum[['zone','distance']].to_csv(r'\avg residuals_all modes\dt_anomaly_cluster/'+key+'_dt_distance_O.csv',index=False)
    disd_sum[['zone','distance']].to_csv(r'\data\avg residuals_all modes\dt_anomaly_cluster/'+key+'_dt_distance_D.csv',index=False)

trans                     zone            datetime  Citibike  FHV  Subway  \
0      Allerton/Pelham Gardens 2021-07-08 00:00:00  0.000000  0.0     0.0   
1      Allerton/Pelham Gardens 2021-07-08 04:00:00  0.000000  0.0     0.0   
2      Allerton/Pelham Gardens 2021-07-08 08:00:00  0.000000  0.0     0.0   
3      Allerton/Pelham Gardens 2021-07-08 12:00:00  0.000000  0.0     0.0   
4      Allerton/Pelham Gardens 2021-07-08 16:00:00  0.000000  0.0     0.0   
...                        ...                 ...       ...  ...     ...   
3115            Yorkville West 2021-07-09 04:00:00  0.000000  0.0     0.0   
3116            Yorkville West 2021-07-09 08:00:00 -2.776756  0.0     0.0   
3117            Yorkville West 2021-07-09 12:00:00  0.000000  0.0     0.0   
3118            Yorkville West 2021-07-09 16:00:00  0.000000  0.0     0.0   
3119            Yorkville West 2021-07-09 20:00:00  0.000000  0.0     0.0   

trans  Taxi  
0       0.0  
1       0.0  
2       0.0  
3       0.0  
4    